In [3]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from utils import file_to_coupling_map, directory_to_circuits

import random
import time
import os
import numpy as np


In [4]:
seed = 42
random.seed(seed)

cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
qc_name = "qft"
heuristic = "decay"
curr_seed = 42
qc_list = directory_to_circuits(f"circuits/{qc_name}")

pass_trials_list = [1, 5]


# CSV file to store the results
csv_path = f"results/trials_{qc_name}.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")




In [5]:
# Vary the pass trials
for pass_trials in pass_trials_list:
    print(f"Running {pass_trials} trials")
    # Run on all circuits
    for qc in qc_list:
        num_qubits = qc.num_qubits
        print(f"    Running {qc.name} on {cmap_name} with seed {curr_seed}")
        # Create the routing pass
        rp = SabreSwap(coupling_map=cmap, heuristic=heuristic, seed=curr_seed, trials=pass_trials)
        lp = SabreLayout(coupling_map=cmap, routing_pass=rp, seed=curr_seed)
        # Create the pass manager
        pm = PassManager([lp, FullAncillaAllocation(cmap), EnlargeWithAncilla(), ApplyLayout(), rp])
        # Execute the pass manager
        start = time.time()
        qc_tr = pm.run(qc)
        end = time.time()
        # Collect the metrics for this trial
        runtime = end - start
        swap_count = qc_tr.count_ops().get("swap", 0)
        depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

        # Write the results to the CSV file
        # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
        with open(csv_path, "a") as f:
            f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
        print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

Running 1 trials
    Running qft_10.qasm on heavy_hex_127.txt with seed 42
            Swap count: 61, Depth: 57, Runtime: 0.06658506393432617
    Running qft_29.qasm on heavy_hex_127.txt with seed 42
            Swap count: 681, Depth: 338, Runtime: 0.37375617027282715
    Running qft_49.qasm on heavy_hex_127.txt with seed 42
            Swap count: 2169, Depth: 815, Runtime: 1.0169780254364014
    Running qft_68.qasm on heavy_hex_127.txt with seed 42
            Swap count: 4176, Depth: 1454, Runtime: 1.963526964187622
    Running qft_88.qasm on heavy_hex_127.txt with seed 42
            Swap count: 7482, Depth: 2102, Runtime: 3.293938159942627
    Running qft_107.qasm on heavy_hex_127.txt with seed 42
            Swap count: 9667, Depth: 2517, Runtime: 4.774640798568726
    Running qft_127.qasm on heavy_hex_127.txt with seed 42
            Swap count: 15679, Depth: 6173, Runtime: 6.907954931259155
Running 5 trials
    Running qft_10.qasm on heavy_hex_127.txt with seed 42
           